### Import external modules

In [ ]:
import gym
import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn.metrics import mean_squared_error
import optuna.visualization as vis

from stable_baselines3 import PPO, A2C, SAC
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecMonitor

### Add mbt-gym to path

In [ ]:
import sys
sys.path.append("../")

In [ ]:
from mbt_gym.agents.BaselineAgents import CarteaJaimungalMmAgent
from mbt_gym.gym.helpers.generate_trajectory import generate_trajectory
from mbt_gym.gym.StableBaselinesTradingEnvironment import StableBaselinesTradingEnvironment
from mbt_gym.gym.TradingEnvironment import TradingEnvironment
from mbt_gym.gym.wrappers import *
from mbt_gym.rewards.RewardFunctions import PnL, CjMmCriterion
from mbt_gym.stochastic_processes.midprice_models import *
from mbt_gym.stochastic_processes.arrival_models import PoissonArrivalModel
from mbt_gym.stochastic_processes.fill_probability_models import ExponentialFillFunction
from mbt_gym.gym.ModelDynamics import LimitOrderModelDynamics
from mbt_gym.gym.helpers.plotting import generate_trajectory, generate_results_table_and_hist, plot_trajectory
from mbt_gym.agents.SbAgent import SbAgent

### Create market making environment

In [ ]:
sigma= 0.1 # constant "volatility" of mid-price process
arrival_rate = 10.0 # lambda 
fill_exponent = 1 # kappa
alpha = 0.001 # terminal inventory penalty (fees of market orders and walking the book)
phi = 0.5 # running inventory penalty parameter

terminal_time = 1.0 # time [0,1]
max_inventory = 3
initial_inventory = (-3,4) # initial inventory will be random integer from {-3,-2,...,2,3}
initial_price = 100

n_steps = int(10 * terminal_time * arrival_rate)
step_size = 1/n_steps

In [ ]:
def get_cj_env(num_trajectories:int = 1):
    timestamps = np.linspace(0, terminal_time, n_steps + 1)
    midprice_model = BrownianMotionMidpriceModel(step_size=1/n_steps,
                                                 num_trajectories=num_trajectories)
    arrival_model = PoissonArrivalModel(intensity=np.array([arrival_rate, arrival_rate]), 
                                        step_size=1/n_steps, 
                                        num_trajectories=num_trajectories)
    fill_probability_model = ExponentialFillFunction(fill_exponent=fill_exponent, 
                                                     step_size=1/n_steps,
                                                     num_trajectories=num_trajectories)
    LOtrader = LimitOrderModelDynamics(midprice_model = midprice_model, arrival_model = arrival_model, 
                                fill_probability_model = fill_probability_model,
                                num_trajectories = num_trajectories)
    reward_function = CjMmCriterion(per_step_inventory_aversion = phi, terminal_inventory_aversion = alpha)
    env_params = dict(terminal_time=terminal_time, 
                      n_steps=n_steps,
                      initial_inventory = initial_inventory,
                      model_dynamics = LOtrader,
                      max_inventory=n_steps,
                      normalise_action_space = False,
                      normalise_observation_space = False,
                      reward_function = reward_function,
                      num_trajectories=num_trajectories)
    return TradingEnvironment(**env_params)

In [ ]:
num_trajectories = 1000
env = ReduceStateSizeWrapper(get_cj_env(num_trajectories))
sb_env = StableBaselinesTradingEnvironment(trading_env=env)

In [ ]:
# Monitor sb_env
sb_env = VecMonitor(sb_env)
# Add directory for tensorboard logging and best model
tensorboard_logdir = "./tensorboard/RL-learning-CJ/"
best_model_path = "./SB_models/RL-best-CJ"

### Train PPO Model

In [ ]:
tensorboard_logdir = "./tensorboard/"
best_model_path = "./SB_models/"
n_eval_episodes = 50
eval_freq = 500_000
total_timesteps = 20_000_000

In [ ]:
PPO_tensorboard_logdir = tensorboard_logdir + "PPO-learning-CJ/"
PPO_best_model_path = best_model_path + "PPO-best-CJ"
PPO_callback_params = dict(eval_env=sb_env, n_eval_episodes=n_eval_episodes, best_model_save_path=PPO_best_model_path, eval_freq = eval_freq,
deterministic=True)

PPO_callback = EvalCallback(**PPO_callback_params)
ppo_model = PPO("MlpPolicy", sb_env, verbose=0, tensorboard_log=PPO_tensorboard_logdir, n_steps= int(n_steps), batch_size= int(n_steps * num_trajectories / 10))

In [ ]:
ppo_model.learn(total_timesteps=total_timesteps, callback = PPO_callback)

### Train A2C Model

In [ ]:
# A2C
A2C_tensorboard_logdir = tensorboard_logdir + "A2C-learning-CJ/"
A2C_best_model_path = best_model_path + "A2C-best-CJ"
A2C_callback_params = dict(eval_env=sb_env, n_eval_episodes=n_eval_episodes, 
                           best_model_save_path=A2C_best_model_path, 
                           deterministic=True)

A2C_callback = EvalCallback(**A2C_callback_params)
a2c_model = A2C("MlpPolicy", sb_env, verbose=0, n_steps = 3, tensorboard_log=A2C_tensorboard_logdir)

In [ ]:
a2c_model.learn(total_timesteps=total_timesteps, callback=A2C_callback)

### Train SAC Model 

In [ ]:
# SAC
SAC_tensorboard_logdir = tensorboard_logdir + "SAC-learning-CJ/"
SAC_best_model_path = best_model_path + "SAC-best-CJ"
SAC_callback_params = dict(eval_env=sb_env, n_eval_episodes=n_eval_episodes, 
                           best_model_save_path=SAC_best_model_path, 
                           deterministic=True)

SAC_callback = EvalCallback(**SAC_callback_params)
sac_model = SAC("MlpPolicy", sb_env, verbose=0, batch_size = 256, tensorboard_log=SAC_tensorboard_logdir)

In [ ]:
sac_model.learn(total_timesteps=total_timesteps, callback=SAC_callback)

### Creating RL Agents

In [ ]:
ppo_agent = SbAgent(ppo_model)
sac_agent = SbAgent(sac_model)
a2c_agent = SbAgent(a2c_model)

In [ ]:
inventories = np.arange(-3, 4, 1)

# Collect PPO actions
ppo_bid_actions, ppo_ask_actions = [], []
for inventory in inventories:
    bid_action, ask_action = ppo_agent.get_action(np.array([[inventory, 0.5]])).reshape(-1)
    ppo_bid_actions.append(bid_action)
    ppo_ask_actions.append(ask_action)

# Collect SAC actions
sac_bid_actions, sac_ask_actions = [], []
for inventory in inventories:
    bid_action, ask_action = sac_agent.get_action(np.array([[inventory, 0.5]])).reshape(-1)
    sac_bid_actions.append(bid_action)
    sac_ask_actions.append(ask_action)

# Collect A2C actions
a2c_bid_actions, a2c_ask_actions = [], []
for inventory in inventories:
    bid_action, ask_action = a2c_agent.get_action(np.array([[inventory, 0.5]])).reshape(-1)
    a2c_bid_actions.append(bid_action)
    a2c_ask_actions.append(ask_action)

### Creating Optimal Agent

In [ ]:
cj_agent = CarteaJaimungalMmAgent(env=get_cj_env())

In [ ]:
# Get the Cartea Jaimungal action
cj_bid_actions = []
cj_ask_actions = []
for inventory in inventories:
    bid_action, ask_action = cj_agent.get_action(np.array([[0,inventory,0.5]])).reshape(-1)
    cj_bid_actions.append(bid_action)
    cj_ask_actions.append(ask_action)

### Plotting Depth vs Inventory Plots for Agent Actions

In [ ]:
# Plotting
plt.plot(inventories, ppo_bid_actions, label="PPO bid", color="g",linewidth=0.8)
plt.plot(inventories, ppo_ask_actions, label="PPO ask", color="g",linewidth=0.8)

plt.plot(inventories, sac_bid_actions, label="SAC bid", color="y", linewidth=0.8)
plt.plot(inventories, sac_ask_actions, label="SAC ask", color="y",linewidth=0.8)

plt.plot(inventories, a2c_bid_actions, label="A2C bid", color="b",linewidth=0.8)
plt.plot(inventories, a2c_ask_actions, label="A2C ask", color="b",linewidth=0.8)

plt.plot(inventories, cj_bid_actions, label="CJ bid", color="k", linestyle="--", linewidth=2.5)
plt.plot(inventories, cj_ask_actions, label="CJ ask", color="r", linestyle="--", linewidth=2.5)
# Adding title and axis labels
plt.title("Depth vs Inventory")
plt.xlabel("Inventory")
plt.ylabel("Depth")
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

### Plotting MSE for Agent Actions

In [ ]:
from sklearn.metrics import mean_squared_error

mse_ppo_bid = mean_squared_error(cj_bid_actions, ppo_bid_actions)
mse_ppo_ask = mean_squared_error(cj_ask_actions, ppo_ask_actions)

# Do the same for SAC and A2C
mse_a2c_bid = mean_squared_error(cj_bid_actions, a2c_bid_actions)
mse_a2c_ask = mean_squared_error(cj_ask_actions, a2c_ask_actions)

mse_sac_bid = mean_squared_error(cj_bid_actions, sac_bid_actions)
mse_sac_ask = mean_squared_error(cj_ask_actions, sac_ask_actions)

In [ ]:
labels = ['PPO', 'A2C', 'SAC']
bid_mse_values = [mse_ppo_bid, mse_a2c_bid, mse_sac_bid]
ask_mse_values = [mse_ppo_ask, mse_a2c_ask, mse_sac_ask]

x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, bid_mse_values, width, label='Bid MSE')
rects2 = ax.bar(x + width/2, ask_mse_values, width, label='Ask MSE')

ax.set_ylabel('MSE')
ax.set_title('MSE values by policy and action type')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

plt.show()

### Optimising PPO Hyperparameters 

In [ ]:
import optuna
from stable_baselines3.common.evaluation import evaluate_policy
def objective_ppo_mse(trial):
    # Define the hyperparameter search space
    gamma = trial.suggest_float("gamma", 0.9, 0.9999)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    vf_coef = trial.suggest_float("vf_coef", 0.1, 1.0)
    ent_coef = trial.suggest_float("ent_coef", 1e-8, 1e-1, log=True)
    clip_range = trial.suggest_float("clip_range", 0.1, 0.4)

    # Create the PPO model with the specified hyperparameters
    ppo_model_mse = PPO("MlpPolicy", sb_env, gamma=gamma, learning_rate=learning_rate, 
                vf_coef=vf_coef, ent_coef=ent_coef, clip_range=clip_range, verbose=0, n_steps= int(n_steps), batch_size= int(n_steps * num_trajectories / 10))
    
    # Train the model
    ppo_model_mse.learn(total_timesteps=20_000_000)
    
    # Create agent and evaluate
    agent = SbAgent(ppo_model_mse)
    bid_actions, ask_actions = [], []
    for inventory in inventories:
        bid_action, ask_action = agent.get_action(np.array([[inventory, 0.5]])).reshape(-1)
        bid_actions.append(bid_action)
        ask_actions.append(ask_action)

    mse = np.mean((np.array(bid_actions) - np.array(cj_bid_actions))**2 +
                  (np.array(ask_actions) - np.array(cj_ask_actions))**2)

    return -mse

# Initiate the study object
study_ppo_mse = optuna.create_study(direction="maximize")  # We want to maximize the objective

# Run optimization
study_ppo_mse.optimize(objective_ppo_mse, n_trials=25)  # Adjust n_trials based on your computational budget

# Print the result
best_params = study_ppo_mse.best_params
best_value = study_ppo_mse.best_value
print(f"Best value: {best_value}\nWith parameters: {best_params}")

### Visualising Optimisation Process

In [ ]:
# Visualization
%matplotlib inline
# 1. Optimization History
vis.plot_optimization_history(study_ppo_mse)

In [ ]:
# 2. Parallel Coordinate Plot
vis.plot_parallel_coordinate(study_ppo_mse)

In [ ]:
# 3. Slice Plot
vis.plot_slice(study_ppo_mse)

In [ ]:
# 4. Contour Plot
# For this, you might want to pick two hyperparameters to visualize, e.g., "gamma" and "learning_rate"
vis.plot_contour(study_ppo_mse, params=["gamma", "ent_coef"])

### Training Model Using Best Hyperparameters from Optimisation Study

In [ ]:
best_model_ppo_mse = PPO("MlpPolicy", sb_env, **best_params, verbose=0, n_steps= int(n_steps), batch_size= int(n_steps * num_trajectories / 10), tensorboard_log=PPO_tensorboard_logdir)

In [ ]:
best_model_ppo_mse.learn(total_timesteps=20_000_000, callback=PPO_callback)  # You can adjust the number of timesteps based on your requirement

### Creating New Agent Using Optimised Model and Plotting Same Plots as Before

In [ ]:
ppo_mse_agent = SbAgent(best_model_ppo_mse)

In [ ]:
# Collect PPO actions
ppo_mse_bid_actions, ppo_mse_ask_actions = [], []
for inventory in inventories:
    bid_action, ask_action = ppo_mse_agent.get_action(np.array([[inventory, 0.5]])).reshape(-1)
    ppo_mse_bid_actions.append(bid_action)
    ppo_mse_ask_actions.append(ask_action)

In [ ]:
plt.plot(inventories, ppo_bid_actions, label="PPO Bid", color="y",linewidth=0.8)
plt.plot(inventories, ppo_ask_actions, label="PPO Ask", color="y",linewidth=0.8)

plt.plot(inventories, ppo_mse_bid_actions, label="PPO MSE Opt. Bid", color="b",linewidth=0.8)
plt.plot(inventories, ppo_mse_ask_actions, label="PPO MSE Opt. Ask", color="b",linewidth=0.8)

plt.plot(inventories, cj_bid_actions, label="CJ Bid", color="k", linestyle="--", linewidth=2.5)
plt.plot(inventories, cj_ask_actions, label="CJ Ask", color="r", linestyle="--", linewidth=2.5)
# Adding title and axis labels
plt.title("Depth vs Inventory")
plt.xlabel("Inventory")
plt.ylabel("Depth")
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
mse_ppo_bid_mse = mean_squared_error(cj_bid_actions, ppo_mse_bid_actions)
mse_ppo_ask_mse = mean_squared_error(cj_ask_actions, ppo_mse_ask_actions)

In [ ]:
labels = ['PPO No Opt.','PPO MSE Opt.']
bid_mse_values = [mse_ppo_bid, mse_ppo_bid_mse]
ask_mse_values = [mse_ppo_ask, mse_ppo_ask_mse]

x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, bid_mse_values, width, label='Bid MSE')
rects2 = ax.bar(x + width/2, ask_mse_values, width, label='Ask MSE')

ax.set_ylabel('MSE')
ax.set_title('MSE values by optimisation method')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

plt.show()

### Performing 2nd Optimisation Study for Architecture Hyperparameters Using Best Hyperparameters from the Last Study

In [ ]:
# Using the best hyperparameters from the previous study
best_params_mse = study_ppo_mse.best_params

PPO_activation_functions_to_test = ['Tanh', 'ReLU', 'Sigmoid', 'LeakyReLU']  # Note the capitalization for torch.nn functions
PPO_num_layers_to_test = [1, 2, 3]
PPO_hidden_units_to_test = [32, 64, 128, 256]

def objective_ppo_architecture(trial):
    # Use the best hyperparameters from the previous optimization
    gamma = best_params_mse["gamma"]
    learning_rate = best_params_mse["learning_rate"]
    vf_coef = best_params_mse["vf_coef"]
    ent_coef = best_params_mse["ent_coef"]
    clip_range = best_params_mse["clip_range"]

    # Now optimize the new parameters
    activation_function = trial.suggest_categorical("activation_function", PPO_activation_functions_to_test)
    n_layers = trial.suggest_categorical("n_layers", PPO_num_layers_to_test)
    hidden_units = trial.suggest_categorical("hidden_units", PPO_hidden_units_to_test)

    layers = [hidden_units] * n_layers  # Repeating the chosen number of units for n_layers times
    
    policy_kwargs = dict(
        net_arch=[dict(pi=layers, vf=layers)],
        activation_fn=getattr(torch.nn, activation_function)
    )

    # Create the PPO model with the specified parameters
    ppo_model_arch = PPO("MlpPolicy", sb_env, gamma=gamma, learning_rate=learning_rate, 
                vf_coef=vf_coef, ent_coef=ent_coef, clip_range=clip_range, verbose=0, 
                n_steps=int(n_steps), batch_size=int(n_steps * num_trajectories / 10),
                policy_kwargs=policy_kwargs)

    # Train the model
    ppo_model_arch.learn(total_timesteps=20_000_000)
    
    # Create agent and evaluate
    agent = SbAgent(ppo_model_arch)
    bid_actions, ask_actions = [], []
    for inventory in inventories:
        bid_action, ask_action = agent.get_action(np.array([[inventory, 0.5]])).reshape(-1)
        bid_actions.append(bid_action)
        ask_actions.append(ask_action)

    mse = np.mean((np.array(bid_actions) - np.array(cj_bid_actions))**2 +
                  (np.array(ask_actions) - np.array(cj_ask_actions))**2)

    return -mse  # Assuming you want to maximize the negative mean squared error again

# New Optuna study
study_ppo_architecture = optuna.create_study(direction="maximize")
study_ppo_architecture.optimize(objective_ppo_architecture, n_trials=25)

# Print the result
best_params_architecture = study_ppo_architecture.best_params
best_value_architecture = study_ppo_architecture.best_value
print(f"Best value: {best_value_architecture}\nWith parameters: {best_params_architecture}")

In [ ]:
# Extract best parameters
best_arch_params = study_ppo_architecture.best_params

# Define architecture based on the best parameters
activation_function = getattr(torch.nn, best_arch_params['activation_function'])
hidden_units = best_arch_params['hidden_units']
n_layers = best_arch_params['n_layers']

# Construct the architecture using the best number of layers and units
layers = [hidden_units] * n_layers

policy_kwargs = dict(
    net_arch=[dict(pi=layers, vf=layers)],
    activation_fn=activation_function
)

# The **all_best_params will unpack and use both sets of parameters.
best_model_ppo_architecture = PPO("MlpPolicy", sb_env, **best_params, policy_kwargs=policy_kwargs, verbose=0, n_steps=int(n_steps), batch_size=int(n_steps * num_trajectories / 10), tensorboard_log=PPO_tensorboard_logdir)

In [ ]:
best_model_ppo_architecture.learn(total_timesteps=20_000_000, callback=PPO_callback)

### Plotting Same Graphs as Before Using the Final Optimised Agent, Agent with no Optimisation, and Optimal Agent

In [ ]:
ppo_arc_agent = SbAgent(best_model_ppo_architecture)

In [ ]:
# Collect PPO actions
ppo_arc_bid_actions, ppo_arc_ask_actions = [], []
for inventory in inventories:
    bid_action, ask_action = ppo_arc_agent.get_action(np.array([[inventory, 0.5]])).reshape(-1)
    ppo_arc_bid_actions.append(bid_action)
    ppo_arc_ask_actions.append(ask_action)

In [ ]:
plt.plot(inventories, ppo_bid_actions, label="PPO Bid", color="y",linewidth=0.8)
plt.plot(inventories, ppo_ask_actions, label="PPO Ask", color="y",linewidth=0.8)

plt.plot(inventories, ppo_arc_bid_actions, label="Optimised PPO Bid", color="b",linewidth=0.8)
plt.plot(inventories, ppo_arc_ask_actions, label="Optimised PPO Ask", color="b",linewidth=0.8)

plt.plot(inventories, cj_bid_actions, label="CJ Bid", color="k", linestyle="--", linewidth=2.5)
plt.plot(inventories, cj_ask_actions, label="CJ Ask", color="r", linestyle="--", linewidth=2.5)
# Adding title and axis labels
plt.title("Depth vs Inventory")
plt.xlabel("Inventory")
plt.ylabel("Depth")
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
arc_ppo_bid_mse = mean_squared_error(cj_bid_actions, ppo_arc_bid_actions)
arc_ppo_ask_mse = mean_squared_error(cj_ask_actions, ppo_arc_ask_actions)

In [ ]:
labels = ['PPO No Opt.','Optimised PPO']
bid_mse_values = [mse_ppo_bid, arc_ppo_bid_mse]
ask_mse_values = [mse_ppo_ask, arc_ppo_ask_mse]

x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, bid_mse_values, width, label='Bid MSE')
rects2 = ax.bar(x + width/2, ask_mse_values, width, label='Ask MSE')

ax.set_ylabel('MSE')
ax.set_title('MSE Before and After Optimisation')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

plt.show()